# Example use cases

In [1]:
from encodings_hnns.encodings import HypergraphEncodings
from encodings_hnns.laplacians import Laplacians
import numpy as np
from brec_analysis.southern_orc_example import southern_orc_example
from brec_analysis.plotting_graphs_and_hgraphs_for_brec import plot_graph_pair, plot_hypergraph_pair
from encodings_hnns.liftings_and_expansions import lift_to_hypergraph

['/Users/pellegrinraphael/anaconda3/envs/clean_repo/lib/python311.zip', '/Users/pellegrinraphael/anaconda3/envs/clean_repo/lib/python3.11', '/Users/pellegrinraphael/anaconda3/envs/clean_repo/lib/python3.11/lib-dynload', '', '/Users/pellegrinraphael/anaconda3/envs/clean_repo/lib/python3.11/site-packages', '/Users/pellegrinraphael/Desktop/Repos_GNN/Hypergraph_Encodings/src']


ModuleNotFoundError: No module named 'hypernetx'

## A - Encoding interace

In [5]:
hg: dict[str, dict | int] = {
    "hypergraph": {
        "yellow": [0, 1, 2, 3],
        "red": [1, 2],
        "green": [2, 4, 5],
        "blue": [3, 4],
    },
    "features": np.array([[1], [1], [1], [1], [1], [1]]),
    "labels": {},
    "n": 6,
}
# Instantiates the Hypergraph Curvature Profile class
hgcurvaturprofile = HypergraphEncodings()
hg = hgcurvaturprofile.add_randowm_walks_encodings(hg, verbose=False)
# k is 20 so the features are shape n by 21
assert isinstance(hg["features"], np.ndarray)
assert hg["features"].shape[0] == hg["n"]
assert hg["features"].shape[1] == 21, f"The shape is {hg['features'].shape[1]}"

hg = hgcurvaturprofile.add_degree_encodings(hg, verbose=True)
assert hg["features"].shape[0] == hg["n"]
assert hg["features"].shape[1] == 27, f"The shape is {hg['features'].shape[1]}"

hg = hgcurvaturprofile.add_laplacian_encodings(hg, verbose=True)
assert hg["features"].shape[0] == hg["n"]
assert hg["features"].shape[1] == 33, f"The shape is {hg['features'].shape[1]}"

# 33 plus 5 makes 38
hg = hgcurvaturprofile.add_curvature_encodings(hg, True)
assert hg["features"].shape[0] == hg["n"]
assert hg["features"].shape[1] == 38, f"The shape is {hg['features'].shape[1]}"

# hg = hgcurvaturprofile.add_laplacian_encodings(hg)
hg = hgcurvaturprofile.add_randowm_walks_encodings(hg)
assert hg["features"].shape[0] == hg["n"]
print(hg)

The hypergraph features for node 0, are 
 [1.         0.         0.14814815 0.08024691 0.10048011 0.08681984
 0.09403895 0.08909179 0.09215521 0.09012237 0.09142979 0.09057234
 0.09112992 0.09076514 0.09100335 0.09084744 0.09094948 0.09088263
 0.09092645 0.0908977  0.09091657]
We add the degree encoding:
 [[1.         2.         3.         2.         2.33333333 0.47140452]]
The stacked features are 
 [[1.         0.         0.14814815 0.08024691 0.10048011 0.08681984
  0.09403895 0.08909179 0.09215521 0.09012237 0.09142979 0.09057234
  0.09112992 0.09076514 0.09100335 0.09084744 0.09094948 0.09088263
  0.09092645 0.0908977  0.09091657 1.         2.         3.
  2.         2.33333333 0.47140452]]
The hypergraph features for node 1, are 
 [1.         0.         0.37962963 0.09259259 0.24879973 0.14366141
 0.20723705 0.16623099 0.19185113 0.17551149 0.18584563 0.17926119
 0.18345126 0.18077571 0.18248561 0.18139043 0.18209288 0.18164153
 0.18193197 0.18174478 0.18186561]
We add the degree

### A-1: Lower level: Laplacian 

In [6]:
hg: dict[str, dict | int] = {
    "hypergraph": {
        "yellow": [1, 2, 3],
        "red": [2, 3],
        "green": [3, 5, 6],
        "blue": [4, 5],
    },
    "features": {},
    "labels": {},
    "n": 6,
}
data = hg

# Instantiates the Laplacians class
laplacian = Laplacians(data)
laplacian.compute_node_neighbors()
print(f"node_neighbors: \n {laplacian.node_neighbors}")
laplacian.compute_node_degrees()
print(f"node_degrees: \n {laplacian.node_degrees}")
laplacian.compute_random_walk_laplacian(rw_type="WE")
laplacian.compute_normalized_laplacian()
laplacian.compute_random_walk_laplacian()
print(f"rw_laplacian: \n {laplacian.rw_laplacian}")

node_neighbors: 
 OrderedDict([(1, {2, 3}), (2, {1, 3}), (3, {1, 2, 5, 6}), (4, {5}), (5, {3, 4, 6}), (6, {3, 5})])
node_degrees: 
 OrderedDict([(1, 1), (2, 2), (3, 3), (4, 1), (5, 2), (6, 1)])
The nodes are [1, 2, 3, 4, 5, 6]
The nodes are [1, 2, 3, 4, 5, 6]
rw_laplacian: 
 [[ 1.         -0.5        -0.5         0.          0.          0.        ]
 [-0.5         1.         -0.5         0.          0.          0.        ]
 [-0.25       -0.25        1.          0.         -0.25       -0.25      ]
 [ 0.          0.          0.          1.         -1.          0.        ]
 [ 0.          0.         -0.33333333 -0.33333333  1.         -0.33333333]
 [ 0.          0.         -0.5         0.         -0.5         1.        ]]
DONE


## B: Brec


In [7]:
# Load the Rook and Shrikhande graphs
G1, G2 = southern_orc_example()  # These are NetworkX graphs

# Plot the original graphs
plot_graph_pair(G1, G2, "rook_vs_shrikhande", "Special", False, "plots/graph_pairs")

# Lift to hypergraphs
hg1_lifted = lift_to_hypergraph(G1, verbose=False, already_in_nx=True)
hg2_lifted = lift_to_hypergraph(G2, verbose=False, already_in_nx=True)

# Plot the lifted hypergraphs
plot_hypergraph_pair(
    G1, G2, hg1_lifted, hg2_lifted, 
    "rook_vs_shrikhande", "Special", False, 
    "plots/hypergraph_pairs"
)

ModuleNotFoundError: No module named 'ot'